In [12]:
import pandas as pd
import tweepy
import csv

In [2]:
keys = pd.read_csv("misllavesapp.csv", header=None)
keys = dict(zip(keys[0],keys[1]))
keys.keys()

dict_keys(['api_key', 'api_secret_key', 'access_token', 'access_token_secret', 'bearer_token'])

In [3]:
# Autenticar a través de la API 
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
# Hacer un tuit

#api.update_status('twit de prueba :rocket: @bello_yasmith  🥺🥺🥺')

In [9]:
# Paso 4
tweets = tweepy.Cursor(api.search,
                       q=['giro de italia -RT'],
                       lang='es',
                       geocode='4.7110,-74.0721,100km',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(100)


In [10]:
# Objeto generador
t = [tweet for tweet in tweets]
t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Bogotá')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    
    t_processed.append(temp)
    
    
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'city', 'user_location', 'id', 'link']
data.head()


,tweets,date,city,user_location,id,link
0,#Entérese #Ciclismo Egan Bernal fue segundo en...,2021-05-14 01:02:35,Bogotá,Bogotá - Colombia,1393008809317212160,https://twitter.com/i/web/stuatus/139300880931...
1,¡Mikel Landa deberá ser operado tras su accide...,2021-05-13 23:58:34,Bogotá,Bogotá,1392992698085388291,https://twitter.com/i/web/stuatus/139299269808...
2,¡Golpea el Ineos! 🇨🇴🔥🚴‍♂️Egan Bernal otra vez ...,2021-05-13 23:00:00,Bogotá,Bogotá. Colombia,1392977960177569793,https://twitter.com/i/web/stuatus/139297796017...
3,#DeportesRCN |VIDEO| 🇨🇴🚴‍♂️\n\nEgan Bernal hab...,2021-05-13 22:45:26,Bogotá,"Bogotá, Colombia",1392974292242112518,https://twitter.com/i/web/stuatus/139297429224...
4,Felicitaciones para el Ciclista colombiano @Eg...,2021-05-13 22:19:56,Bogotá,"Bogotá, D.C., Colombia",1392967876970229767,https://twitter.com/i/web/stuatus/139296787697...


In [14]:
data.csv("archivo.csv", index=False)

AttributeError: 'DataFrame' object has no attribute 'csv'

In [16]:
import snscrape.modules.twitter as sntwitter
import snscrape.modules.twitter as sntwitter


In [20]:
maxTweets = 20 
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#eragoldeyepes' + 'since:2014-07-15 until:2014-08-15').get_items()) :
        if i > maxTweets :
            break
        print(tweet)
        print(tweet.date)
        print(tweet.content)
        print("*"*100)
        print()
        
        
data = data[['tweets', 'date']]
data.head()


,tweets,date
0,#Entérese #Ciclismo Egan Bernal fue segundo en...,2021-05-14 01:02:35
1,¡Mikel Landa deberá ser operado tras su accide...,2021-05-13 23:58:34
2,¡Golpea el Ineos! 🇨🇴🔥🚴‍♂️Egan Bernal otra vez ...,2021-05-13 23:00:00
3,#DeportesRCN |VIDEO| 🇨🇴🚴‍♂️\n\nEgan Bernal hab...,2021-05-13 22:45:26
4,Felicitaciones para el Ciclista colombiano @Eg...,2021-05-13 22:19:56


In [ ]:
import re 

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions
  
  
data[['texto_limpio', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data


import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list
  
  

data['emojis'] = data['texto_limpio'].apply(lambda text: get_emojis(text))
data.head()


data['hora'] = data['date'].dt.floor('T').dt.time
temp = pd.DataFrame(data.hora.value_counts()).reset_index()
temp.columns = ['hora', 'cnt']
temp = temp.sort_values('hora')
temp


import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)


trace = go.Scatter(x=temp.hora.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.hora.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Horas',
                   line={'color': 'blue'})

layout = go.Layout(title="Número de tuits por hora")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)



import scipy.cluster.hierarchy as sch
import numpy as np

pairwise_distances = sch.distance.pdist(temp)
linkage = sch.linkage(pairwise_distances, method='ward')
idx_to_cluster_array = sch.fcluster(linkage, pairwise_distances.max() * 0.5, criterion='distance')
idx = np.argsort(idx_to_cluster_array)
temp = temp.copy()
    
temp2 = temp.iloc[idx, :].T.iloc[idx, :]
my_idx = idx_to_cluster_array

temp2.index = ['tweet #'+str(i) for i in temp2.columns]
temp2.columns = temp2.index


trace = go.Heatmap(z=temp2.values.tolist(),
                   x=temp2.index.values,
                   y=temp2.columns.values,
                   colorscale='Oranges')

layout = go.Layout(title='🔥 Mapa de calor entre tweets (organizado) 🔥',
                   width=800, height=800)

fig = go.Figure(data=[trace],layout=layout)
iplot(fig)




data['cluster'] = my_idx 

for c in data.cluster.value_counts().index:
    df_temp = data[data.cluster==c]
    
    sim = temp2[['tweet #'+str(i) for i in df_temp.index]].mean().mean()
    print(f"***** El cluster {c} tiene {df_temp.shape[0]} tweets con una similaridad de {sim:.2%} *****\n")
    
    tweets_ejemplo = df_temp.head().tweets.values
    for te in tweets_ejemplo:
        print("TWEET")
        print(te)
        print()
    print()
    